In [1]:
!pip install datasets peft -qq
!pip install accelerate -qq
!pip install bitsandbytes -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
from huggingface_hub import login
login()

# Access Key for llama Model (HuggingFace)



from datasets import load_dataset
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    Trainer,
    BitsAndBytesConfig)

from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [3]:
# Used Gemma 2 Foundation Model

model_name = "meta-llama/Llama-3.2-1B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
    )


tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    )

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# model.config.pretraining_tp = 1
# model.config.use_cache = False


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [4]:
print(f"meta-llama/Llama-3.2-1B:\n{base_model}")

meta-llama/Llama-3.2-1B:
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm

In [5]:
print(f"{base_model.config}")

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "meta-llama/Llama-3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bi

In [6]:
def trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    return f"- Trainable model parameters: {trainable_params}.\n- All model parameters: {all_param}.\n- Percentage of trainable model parameters: {100 * trainable_params / all_param:.2f}%"

print(trainable_parameters(base_model))

- Trainable model parameters: 262735872.
- All model parameters: 749275136.
- Percentage of trainable model parameters: 35.07%


In [7]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [8]:
model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(model, lora_config)
print("After PEFT wrapping:")
print(trainable_parameters(model))


After PEFT wrapping:
- Trainable model parameters: 851968.
- All model parameters: 750127104.
- Percentage of trainable model parameters: 0.11%


In [12]:
from datasets import concatenate_datasets

scraped_dataset = load_dataset("json", data_files="./scraped_data.json")
# processed_dataset = load_dataset("json", data_files="./processed_docs.json")
# If the JSON files do not include splits, they will be loaded as the 'train' split by default.
merged_dataset = concatenate_datasets([scraped_dataset["train"], processed_dataset["train"]])
print(f"Total examples in merged dataset: {len(merged_dataset)}")



Total examples in merged dataset: 160


In [18]:
scraped_dataset['train'][0]

{'url': 'https://www.healthsystemtracker.org/chart-collection/quality-u-s-healthcare-system-compare-countries/',
 'title': 'How does the quality of the U.S. health system compare to other countries? - Peterson-KFF Health System Tracker',
 'paragraphs': ['ByEmma WagerTwitter,Imani Telesford,Shameek Rakshit,Nisha Kurani, andCynthia CoxKFF',
  'October 9, 2024',
  'Despite spending nearlytwice as muchper capita on healthcare compared to similarly large and wealthy nations, the United States has a lower life expectancy than peer nations and has seen worsening measures of health outcomes since the COVID-19 pandemic.',
  'This chart collection combines various measures of quality of care in the United States and other large, high-income nations (based on gross domestic product and per capita GDP) to show how the U.S. stacks up against its peers and how that has changed over time.',
  'Generally, the U.S. performs worse in long-term health outcomes measures (such as life expectancy), certain 

In [20]:
def tokenize_function(examples):
    texts = []
    for title, paragraphs in zip(examples["title"], examples["paragraphs"]):
        # Replace None with empty string if necessary
        title = title if title is not None else ""
        paragraphs = paragraphs if paragraphs is not None else []
        combined_text = title + "\n" + "\n".join(paragraphs)
        texts.append(combined_text)
    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = merged_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["url", "title", "paragraphs", "links"])
tokenized_dataset.set_format(type="torch")
print("Tokenization complete.")


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Tokenization complete.


In [21]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./llama-3.2-1B-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=50,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
# Split the tokenized dataset into training and evaluation sets (e.g., 90% train, 10% eval)
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# Create the Trainer with both train and eval datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

print("Starting training...")
trainer.train()
print("Training complete.")


Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


Training complete.


In [24]:
eval_results = trainer.evaluate()
print("Evaluation Results:")
print(eval_results)


Evaluation Results:
{'eval_loss': 2.109065532684326, 'eval_runtime': 1.6805, 'eval_samples_per_second': 9.521, 'eval_steps_per_second': 1.19, 'epoch': 3.0}


In [28]:
def generate_text(prompt, max_length=100, temperature=0.7, top_p=0.95):
    # Tokenize the prompt and explicitly return an attention mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    # Ensure the inputs (including attention_mask) are moved to the correct device
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    # Generate text using both input_ids and attention_mask
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Test the generation function:
# sample_prompt = "Explain the impact of fine-tuning on the performance of language models. Does fine-tuning improve the performance of a language model? Explain."
prompt = "Using the dataset from the Peterson-KFF Health System Tracker on U.S. healthcare quality, provide a comprehensive analysis comparing the United States to other high-income countries. In your response, summarize key metrics such as life expectancy, all-cause mortality, maternal mortality, and rates of premature death. Discuss the impact of socioeconomic factors and healthcare utilization on these outcomes, and explain why the U.S. may perform worse on several indicators despite high per capita spending."
print(generate_text(prompt, max_length=512))


Using the dataset from the Peterson-KFF Health System Tracker on U.S. healthcare quality, provide a comprehensive analysis comparing the United States to other high-income countries. In your response, summarize key metrics such as life expectancy, all-cause mortality, maternal mortality, and rates of premature death. Discuss the impact of socioeconomic factors and healthcare utilization on these outcomes, and explain why the U.S. may perform worse on several indicators despite high per capita spending. Provide a detailed analysis of at least one of the following metrics:
Discuss the impact of socioeconomic factors and healthcare utilization on these outcomes. Provide a detailed analysis of at least one of the following metrics:
Mortality rates: Life expectancy, all-cause mortality, and maternal mortality
Healthcare utilization: Number of hospital visits, total healthcare spending, and outpatient visits
Quality of care: Hospital mortality rate and healthcare provider quality
Socioeconom